In [2]:
import numpy as np
import cv2 
import pandas as pd 
import rasterio
import os

In [3]:
def scale_to_uint8(arr, arr_min = 0.1, arr_max = 0.7):
    arr_scaled = (arr - arr_min) / (arr_max - arr_min)
    arr_scaled = np.clip(arr_scaled, 0, 1)
    arr_uint8 = (arr_scaled * 255)
    arr_uint8[np.isnan(arr)] = 0  # Set NaN to 0
    return arr_uint8.astype(np.uint8)

# Save as green channel PNGs
def save_green_png(arr_uint8, path, scale, mask = None, is_ground_truth = False):
    h, w = arr_uint8.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)
    rgb[..., 1] = arr_uint8
    # Resize the image to 8x larger using nearest neighbor for clear pixel boundaries
    rgb_large = cv2.resize(rgb, (w * scale, h * scale), interpolation=cv2.INTER_NEAREST)
    if mask is not None:
        if mask.shape != rgb_large.shape[:2]:
            raise ValueError("Mask shape does not match resized image shape")
        # Apply mask: set all non-agriculture (mask==0) pixels to black
        rgb_large[mask == 0] = 0

    if is_ground_truth:
        gray = rgb_large[:,:,1]
        zero_mask = (gray == 0).astype(np.uint8)*255

        # Find contours surrouding area that has no data 
        contours, _ = cv2.findContours(zero_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(rgb_large, contours, -1, (0, 0, 255), 3)
    cv2.imwrite(path, rgb_large)

In [8]:
with rasterio.open('roi_inference/songcuulong/data/s1_images/S1_2021-01-14.tif') as src:
    profile = src.profile 
    transform = src.transform 
    crs = src.crs 
    image_shape = src.shape

profile.update(
    {
        'count' : 1,
        'dtype' : 'float32',
    }
)

In [10]:
target_dates = []
combination_folder = 'roi_inference/songcuulong/csv_output/combination'
target_dates = []
for s1_file in os.listdir(combination_folder):
    date = s1_file.split('.')[0].split('_')[0]
    target_dates.append(date)
scale = 4
arr_min = 0.1
arr_max = 0.7 
mask = None 
for target_date in target_dates:
    predicted = pd.read_csv(f'roi_inference/songcuulong/prediction/{target_date}.csv')
    predicted_values = np.array(predicted['Prediction']).reshape(image_shape)
    pred = predicted_values[(predicted_values > 0) & (predicted_values < 0.7)]
    print(f"Predicted values mean ({target_date}): {np.nanmean(pred)}")
    output_path = os.path.join('roi_inference/songcuulong/sm',f'{target_date}_1.tif')
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(predicted_values.astype('float32'), 1)

Predicted values mean (2021-04-14): 0.4686314681109762
Predicted values mean (2020-07-14): 0.4448353046798763
Predicted values mean (2020-09-05): 0.45462778882769117
Predicted values mean (2021-04-28): 0.4898612205830755
Predicted values mean (2020-03-08): 0.41736063624290254
Predicted values mean (2020-03-14): 0.4255809942260249
Predicted values mean (2020-06-18): 0.45792175179778355
Predicted values mean (2020-06-20): 0.4559259670166817
Predicted values mean (2021-11-28): 0.5214914794930885
Predicted values mean (2020-10-04): 0.5080311028807221
Predicted values mean (2020-12-10): 0.5279202001147365
Predicted values mean (2020-02-14): 0.4084869345839357
Predicted values mean (2020-03-26): 0.4434469292959236
Predicted values mean (2020-05-08): 0.4316100950305227
Predicted values mean (2020-01-20): 0.4315877940795708
Predicted values mean (2021-08-24): 0.4617672563396248
Predicted values mean (2020-08-12): 0.41938386157599505
Predicted values mean (2021-10-29): 0.5367067187984101
Predic

KeyboardInterrupt: 

### Visualize Prediction

In [ ]:
target_dates = ['2021-01-14','2021-02-15','2021-03-03','2021-04-04','2021-04-20','2021-05-20',
                '2021-05-22','2021-07-07','2021-07-09','2021-08-26','2021-10-13','2021-11-28',
                '2021-07-25','2021-08-24','2021-10-11','2021-10-29','2021-11-30','2021-12-16']

target_dates = ['2021-01-29','2021-02-22','2021-03-28','2021-05-05','2021-07-08','2021-08-09',
               '2021-09-26','2021-10-30']

scale = 2
arr_min = 0.1
arr_max = 0.7 
mask = None 
image_shape = (140, 140)
for target_date in target_dates:
    predicted = pd.read_csv(f'roi_inference/songcuulong/prediction/{target_date}.csv')
    predicted_values = np.array(predicted['Prediction']).reshape(image_shape)
    g_min = np.nanmin(predicted_values)
    g_max = max(np.nanmax(predicted_values), 0.7)
    pred_uint8 = scale_to_uint8(predicted_values, g_min, g_max)
    pred = predicted_values[(predicted_values > 0) & (predicted_values < 0.7)]
    print(f"Predicted values mean: {np.nanmean(pred)}")
    save_green_png(pred_uint8, f'/mnt/data2tb/Transfer-DenseSM-E_2/roi_inference/songcuulong/output/{target_date}.png', scale, mask)

Predicted values mean: 0.28377616656885923
Predicted values mean: 0.2587036497394655
Predicted values mean: 0.24646114656398105
Predicted values mean: 0.31351878339671124
Predicted values mean: 0.3657872538186023
Predicted values mean: 0.3813019685421377
Predicted values mean: 0.4238395246294964
Predicted values mean: 0.4256401350659318


In [ ]:
target_date = '2021-12-15'
target_dates = ['2021-01-13','2021-02-14','2021-03-02','2021-04-03','2021-04-19','2021-05-21',
                '2021-07-08','2021-08-25','2021-10-12','2021-07-24','2021-10-28','2021-11-29',
                '2021-12-15']
np.set_printoptions(threshold = np.inf)
for target_date in target_dates: 
    with rasterio.open(f'roi_inference/songcuulong/output/tif/{target_date}.tif') as src:
        band1 = src.read(1)
        band2 = src.read(2)
        band1 = np.where((band1 == 0.0) & (band1 > 0.8), np.nan, band1)
        band2 = np.where((band2 == 0.0) & (band2 > 0.8), np.nan, band2)
        ground_truth = 0.5*(band1 + band2)
        print(src.shape)
    

    print(f"Ground truth values mean for {target_date}: {np.nanmean(ground_truth)}")
    g_min = np.nanmin(ground_truth)
    g_max = max(np.nanmax(ground_truth), 0.7)
    gt_uint8 = scale_to_uint8(ground_truth, g_min, g_max)
    

    save_green_png(gt_uint8, f'/mnt/data2tb/Transfer-DenseSM-E_2/roi_inference/songcuulong/output/{target_date}_gt.png', 2, mask = None, is_ground_truth=True)

(175, 308)
Ground truth values mean for 2021-01-13: 0.46745336055755615
(175, 308)
Ground truth values mean for 2021-02-14: 0.43579572439193726
(175, 308)
Ground truth values mean for 2021-03-02: 0.41873666644096375
(175, 308)
Ground truth values mean for 2021-04-03: 0.4752611517906189
(175, 308)
Ground truth values mean for 2021-04-19: 0.5314743518829346
(175, 308)
Ground truth values mean for 2021-05-21: 0.5279948115348816
(175, 308)
Ground truth values mean for 2021-07-08: 0.4653916358947754
(175, 308)
Ground truth values mean for 2021-08-25: 0.4732256829738617
(175, 308)
Ground truth values mean for 2021-10-12: 0.5359135866165161
(175, 308)
Ground truth values mean for 2021-07-24: 0.4431758224964142
(175, 308)
Ground truth values mean for 2021-10-28: 0.5224831104278564
(175, 308)
Ground truth values mean for 2021-11-29: 0.5392063856124878
(175, 308)
Ground truth values mean for 2021-12-15: 0.5100354552268982


In [ ]:
import cv2 
import numpy as np 
img = cv2.imread(f'/mnt/data2tb/Transfer-DenseSM-E_2/roi_inference/songcuulong/output/{target_date}.png')
if img is None:
    print("Image not found")
else:
    h, w, c = img.shape 
    block_size = 100
    h_trim = h - (h % block_size)
    w_trim = w - (w % block_size)
    img_cropped = img[:h_trim, :w_trim]

    # Reshape and compute mean over each block
    img_blocks = img_cropped.reshape(
        h_trim // block_size, block_size, 
        w_trim // block_size, block_size, 
        c
    ) 

    mean_blocks = img_blocks.mean(axis = (1, 3)).astype(np.uint8)
    save_blocks = cv2.resize(mean_blocks, (w, h), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(f'/mnt/data2tb/Transfer-DenseSM-E_2/roi_inference/songcuulong/output/{target_date}_1km.png', save_blocks)
